# Assumptions, notes, (and crisises)

1. Maybe the biggest assumption we have is that people are entering their data correctly and taking the right amount of points. No one would ever lie on 8a.... [Mason](https://www.8a.nu/user/mason-caiby)
1. We're assuming that equally dedicated and motivated and genetically potentialled people are route climbing and bouldering (i.e. Route climbers aren't inheritantly better than boulderererers, even if they are).
2. We're assuming that the points gained by flashing vs onsighting are fair to both boulderers and sport climbers (some might argue one discipline is easier to 1st go than another). This also means that we're assuming flashing a route is about 1 letter grade harder than sending a route (I'd personally disagree) and that onsighting is 3 letter grades harder than flashing a route (this is really area dependent)
3. We're assuming that points gained are season independent. This is probably wrong. Ideally everyone is getting stronger all the time, if the Bubble, CO is anything to go off of, more people are bouldering outside in the winter than in the summer, so bouldering grades might be a little inflated. But, since we're doing worldwide, maybe the bouldering vs route season averages out?
4. We're assuming consistent popularity of bouldering and sport climbing throughout the world. This is tied to assumption number 1
5. We're assuming both disciplines pursue grades with equal vigor. If boulderers care more about grades than route climbers their score will be higher than sport climbers.
6. We're assuming equal tactics for sport climbers and boulderers. Essentially, that all climbers are trying to maximize their 8a.nu score, spending about a month per project and onsighting as hard as they can. Rifle climbers typically spend a season working a few routes, boulderers (where I'm from) seem to try a handleful of boulders at the same time. This means that boulderers probably send more 'limit' boulders than sport climbers?
7. Ok, look, I know BMI is a bad way to gauge someone's health. But it feels like an adequate way to compare muscle mass per height, right? Like, I'm assuming the top climbers are all relatively as lean as each other (like sub 10% bf for men?) and I want to know if being a Bufffff Boiiii (tm) is better than being a Skinni Salammi (also, tm). Well, really what I want to know is if getting my TinderBod 2019 is worth the risk of not climbing as hard this year. Existentially, is it worth trying to climb harder? When do the basic girls that swipe right for me stop caring about how hard I climb? When do I stop caring about how hard I climb. When will I love myself?
8. Also, sorry. The BMI webscraper thing only kinda works. It seems like a hassle to get non-ascii characters included in the URL (or is it an IRI?). If someone wants to go for an improvement, great. You can send me a pull request, or just do it on your own thing. I literally dont care at all.
9. Also, the [8a robots.txt](https://www.8a.nu/robots.txt) webpage doesn't really discuss the `.../user/...` pages so I just kinda went for it. Please don't sue me. 

10. Scrapping the height/weight data is taking forever. It seems like 3 seconds per person, so... 3*3000 = 9000s = 150 minutes = 2.5 hours.

In [ ]:
%matplotlib inline
# -*- coding: utf-16 -*-
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
import pickle

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

sns.set(palette="muted")

# webscraping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
import urllib3
import urllib.parse
from requests import get
import html5lib
import re
import os


In [ ]:
df = pd.read_csv('data.csv', index_col='ranking')
df = df.replace(r' ', '', regex=True).astype(int)
df.plot()

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_xlabel('Ranking')
ax1.set_ylabel('Sport', color='r')
routes = ax1.plot(df['sport'], color='r')
ax1.tick_params(axis='y', labelcolor='r')
y1_locs = range(9500, 14000, 500)
y1_labels = ['7C+', '8A','8A+', '8B', '8B+', '8C', '8C+', '9A' ,'9A+', '9B']
ax1.set_ylim(9500,14000)
plt.yticks(ticks=y1_locs, labels=y1_labels)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color2 = 'b'
ax2.set_ylabel('Boulders', color=color2)  # we already handled the x-label with ax1
boulders = ax2.plot(df['Boulders'], color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
ax2.set_ylim(8000,12500)
plt.yticks()
y2_locs = [loc for loc in range(8000, 12500, 500)]
y2_labels = ['v7','v8', 'v9', 'v10', 'v11', 'v12','v13','v14','v15', 'v16', 'v17', 'v18']
plt.yticks(ticks=y2_locs, labels = y2_labels)

plt.legend(routes+boulders, ['routes', 'boulders'])

plt.title('Grade Comparison of Routes and Boulders by Ranking')

<table cellspacing="0" rules="all" border="0" id="ctl00_ContentPlaceholder_GridViewRankingRoute" style="border-width:0px;width:415px;border-collapse:collapse;">
			<tbody><tr>
				<td>1</td><td>&nbsp;</td><td><a href="/scorecard/adam-ondra/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=ea0fb3b90e4b0b655580384e07974b38">13&nbsp;835</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/adam-ondra">Adam Ondra</a></td><td>1993</td><td><a href="/cz/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CZE</a></td><td><a id="CityUrl" href="/cz/scorecard/ranking/?City=Brno&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Brno</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>2</td><td>&nbsp;</td><td><a href="/scorecard/piotr-schab/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=286c5cca5e7ef1c7d9b5340996072597">13&nbsp;460</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/piotr-schab">Piotr Schab</a></td><td>1996</td><td><a href="/pl/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">POL</a></td><td><a id="CityUrl" href="/pl/scorecard/ranking/?City=Krak%c3%b3w&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Kraków</a></td>
			</tr><tr>
				<td>3</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/jonathan-siegrist/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=c52d362c88b707d15dfae04b71ffe806">13&nbsp;145</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jonathan-siegrist">Jonathan Siegrist</a></td><td>1985</td><td><a href="/us/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">USA</a></td><td><a id="CityUrl" href="/us/scorecard/ranking/?City=Las+Vegas&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Las Vegas</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>4</td><td>&nbsp;</td><td><a href="/scorecard/pirmin-bertle/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=23235f3cf122d83acc07965e3969d656">13&nbsp;030</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/pirmin-bertle">Pirmin Bertle</a></td><td>1985</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Munich&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Munich</a></td>
			</tr><tr>
				<td>5</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/jorge-diaz-rullo/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=0e0b7d441149c298d42a575a323fbf13">12&nbsp;903</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jorge-diaz-rullo">Jorge diaz-rullo</a></td><td>1999</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=madrid&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">madrid</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>6</td><td>&nbsp;</td><td><a href="/scorecard/stefano-ghisolfi/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=d19aa419e3538de3745ffeebac40908e">12&nbsp;813</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/stefano-ghisolfi">Stefano Ghisolfi</a></td><td>1993</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Arco&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Arco</a></td>
			</tr><tr>
				<td>7</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/mathieu-bouyoud/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=d26a2e6b8908dc34f69df5aec69f30ef">12&nbsp;810</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/mathieu-bouyoud">Mathieu Bouyoud</a></td><td>1989</td><td><a href="/fr/scorecard/classement/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">FRA</a></td><td><a id="CityUrl" href="/fr/scorecard/classement/?City=Chamb%c3%a9ry&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Chambéry</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>8</td><td>&nbsp;</td><td><a href="/scorecard/david-firnenburg/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=4183ecebd277d87d189e1aecd570f6c1">12&nbsp;745</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/david-firnenburg">David Firnenburg</a></td><td>1995</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Zurich&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Zurich</a></td>
			</tr><tr>
				<td>9</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/alex-garriga/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=4e9dfecd07dfd5eb16a8966b169506b5">12&nbsp;500</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/alex-garriga">Alex Garriga</a></td><td>1998</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=Cuenca&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Cuenca</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>10</td><td>&nbsp;</td><td><a href="/scorecard/moritz-welt/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=25bd3f954db07fe0b72726e40c0d05e8">12&nbsp;464</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/moritz-welt">Moritz Welt</a></td><td>2001</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Wannbach&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Wannbach</a></td>
			</tr><tr>
				<td>11</td><td>&nbsp;</td><td><a href="/scorecard/ramon-julian-puigblanque/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=4c2b2bc7feaaeb7cdc711a47fac641c9">12&nbsp;455</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/ramon-julian-puigblanque">Ramón Julian Puigblanque</a></td><td>1981</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=Manlleu&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Manlleu</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>12</td><td>&nbsp;</td><td><a href="/scorecard/thibault-lair/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=39a392d12a507836bc2192d7e6d43f52">12&nbsp;430</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/thibault-lair">Thibault Lair</a></td><td>1992</td><td><a href="/fr/scorecard/classement/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">FRA</a></td><td><a id="CityUrl" href="/fr/scorecard/classement/?City=tarbes&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">tarbes</a></td>
			</tr><tr>
				<td>13</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/luca-bana/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=cf30a351587be1c34cce804ad25e6f19">12&nbsp;393</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/luca-bana">Luca Bana</a></td><td>1998</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Premolo&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Premolo</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>14</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/philipp-gassner/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=4fc65fe2657c71a33c3cd0b236f0a63e">12&nbsp;370</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/philipp-gassner">Philipp Gaßner</a></td><td>2000</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Munich&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Munich</a></td>
			</tr><tr>
				<td>15</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/marco-zanone/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=2ab07a719018fb3356e15cf05b040d46">12&nbsp;350</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/marco-zanone">Marco Zanone</a></td><td>1994</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Ronco+Biellese&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Ronco Biellese</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>16</td><td>&nbsp;</td><td><a href="/scorecard/jan-hojer/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=0f8e2bdd912aa0aa5379110e7d0b932e">12&nbsp;338</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jan-hojer">Jan Hojer</a></td><td>1992</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=K%c3%b6ln&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Köln</a></td>
			</tr><tr>
				<td>17</td><td>&nbsp;</td><td><a href="/scorecard/thomas-p-ohalloran/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=ebc6dfd808ca2dc4b774ed49fea5b5d3">12&nbsp;310</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/thomas-p-ohalloran">Thomas P. O'Halloran</a></td><td>1992</td><td><a href="/au/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">AUS</a></td><td><a id="CityUrl" href="/au/scorecard/ranking/?City=Blackheath&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Blackheath</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>18</td><td>&nbsp;</td><td><a href="/scorecard/peter-dawson/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=efdf30015fadadad19e0574f430d5f57">12&nbsp;290</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/peter-dawson">Peter Dawson</a></td><td>1998</td><td><a href="/uk/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">GBR</a></td><td><a id="CityUrl" href="/uk/scorecard/ranking/?City=Exeter&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Exeter</a></td>
			</tr><tr>
				<td>19</td><td>&nbsp;</td><td><a href="/scorecard/jernej-kruder/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=75141f1fbb4b02f21d23648430946f5d">12&nbsp;287</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jernej-kruder">Jernej Kruder</a></td><td>1990</td><td><a href="/si/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">SVN</a></td><td><a id="CityUrl" href="/si/scorecard/ranking/?City=Celje&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Celje</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>20</td><td>&nbsp;</td><td><a href="/scorecard/luke-dawson/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=06e7bf669c5cedea6266f70d1cc5dd33">12&nbsp;245</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/luke-dawson">Luke Dawson</a></td><td>1996</td><td><a href="/uk/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">GBR</a></td><td><a id="CityUrl" href="/uk/scorecard/ranking/?City=Sheffield&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Sheffield</a></td>
			</tr><tr>
				<td>21</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/matteo-menardi/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=33f585495d440bcf2b18534aa28a6aef">12&nbsp;240</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/matteo-menardi">Matteo Menardi</a></td><td>1998</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Cortina+d%27+Ampezzo&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Cortina d' Ampezzo</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>22</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/evan-hau-27079/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=496d43d6babf73b13515c4e9755266fc">12&nbsp;215</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/evan-hau-27079">Evan Hau</a></td><td>1986</td><td><a href="/ca/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CAN</a></td><td><a id="CityUrl" href="/ca/scorecard/ranking/?City=Calgary&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Calgary</a></td>
			</tr><tr>
				<td>23</td><td>&nbsp;</td><td><a href="/scorecard/matteo-gambaro/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=1b8401c6546c27e3415d706833edc3eb">12&nbsp;215</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/matteo-gambaro">Matteo Gambaro</a></td><td>1974</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1"></a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>24</td><td>&nbsp;</td><td><a href="/scorecard/lukasz-dudek/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=94d4daa949ed3e0e74e0a4725616ebea">12&nbsp;210</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/lukasz-dudek">Lukasz Dudek</a></td><td>1983</td><td><a href="/pl/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">POL</a></td><td><a id="CityUrl" href="/pl/scorecard/ranking/?City=Czestochowa&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Czestochowa</a></td>
			</tr><tr>
				<td>25</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/alexander-rohr/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=aaaed69a5e286cd1590c5dc920454724">12&nbsp;200</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/alexander-rohr">Alexander Rohr</a></td><td>1995</td><td><a href="/ch/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CHE</a></td><td><a id="CityUrl" href="/ch/scorecard/ranking/?City=Rubigen&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Rubigen</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>26</td><td>&nbsp;</td><td><a href="/scorecard/daniel-fuertes/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=acb37e65b0ae6ad2920c9e15b1bb1daa">12&nbsp;198</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/daniel-fuertes">Daniel Fuertes</a></td><td>1980</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=Zaragoza&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Zaragoza</a></td>
			</tr><tr>
				<td>27</td><td>&nbsp;</td><td><a href="/scorecard/jim-pope/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=55b98e2dbc9f81b58a7567940ded2ea5">12&nbsp;195</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jim-pope">Jim Pope</a></td><td>1998</td><td><a href="/uk/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">GBR</a></td><td><a id="CityUrl" href="/uk/scorecard/ranking/?City=London&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">London</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>28</td><td>&nbsp;</td><td><a href="/scorecard/marvin-winkler/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=caf2c9c246cbfd113f2411e6e03b7ba7">12&nbsp;192</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/marvin-winkler">Marvin Winkler</a></td><td>&nbsp;</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=----&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">----</a></td>
			</tr><tr>
				<td>29</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/luis-rodriguez-martin/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=49f2338d920335dcdd9b47e41fbb0427">12&nbsp;190</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/luis-rodriguez-martin">luis rodriguez martin</a></td><td>1988</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=malaga&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">malaga</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>30</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/jakub-kovacik/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=de4cc0e9cc5ace9943fce79854af314a">12&nbsp;180</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jakub-kovacik">Jakub Kováčik</a></td><td>1989</td><td><a href="/sk/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">SVK</a></td><td><a id="CityUrl" href="/sk/scorecard/ranking/?City=Pezinok&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Pezinok</a></td>
			</tr><tr>
				<td>31</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/dalton-bunker/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=b84471e6ae2443e967736f2c977b306e">12&nbsp;177</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/dalton-bunker">Dalton Bunker</a></td><td>1997</td><td><a href="/us/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">USA</a></td><td><a id="CityUrl" href="/us/scorecard/ranking/?City=Orem&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Orem</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>32</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/marcello-bombardi/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=e7a06ccef5de394c92e79c1f8b8219f5">12&nbsp;148</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/marcello-bombardi">Marcello Bombardi</a></td><td>1993</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Casinalbo+(MO)&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Casinalbo (MO)</a></td>
			</tr><tr>
				<td>33</td><td>&nbsp;</td><td><a href="/scorecard/fedir-samoilov/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=ae04a93977bd836f6b3d9f027cd3e196">12&nbsp;140</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/fedir-samoilov">Fedir Samoilov</a></td><td>1997</td><td><a href="/ua/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">UKR</a></td><td><a id="CityUrl" href="/ua/scorecard/ranking/?City=Ukrainka&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Ukrainka</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>34</td><td>&nbsp;</td><td><a href="/scorecard/moritz-perwitzschky/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=cfca30eb7d82be731887d624f4b1770a">12&nbsp;140</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/moritz-perwitzschky">Moritz Perwitzschky</a></td><td>1999</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Igensdorf&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Igensdorf</a></td>
			</tr><tr>
				<td>35</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/peter-kuric/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=531fb8b91c66383147000f8330d18536">12&nbsp;105</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/peter-kuric">Peter Kuric</a></td><td>2001</td><td><a href="/sk/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">SVK</a></td><td><a id="CityUrl" href="/sk/scorecard/ranking/?City=Handlova&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Handlova</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>36</td><td>&nbsp;</td><td><a href="/scorecard/domen-skofic/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=38c3d925fe6dc52867baf3011f051fe2">12&nbsp;088</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/domen-skofic">Domen Škofic</a></td><td>1994</td><td><a href="/si/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">SVN</a></td><td><a id="CityUrl" href="/si/scorecard/ranking/?City=Radovljica&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Radovljica</a></td>
			</tr><tr>
				<td>37</td><td>&nbsp;</td><td><a href="/scorecard/obed-hardmeier/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=0ac6fec60c1a8e9ba3a4fd9277aef52a">12&nbsp;084</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/obed-hardmeier">Obed Hardmeier</a></td><td>1994</td><td><a href="/ch/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CHE</a></td><td><a id="CityUrl" href="/ch/scorecard/ranking/?City=Hinteregg&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Hinteregg</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>38</td><td>&nbsp;</td><td><a href="/scorecard/lucien-martinez/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=5aef58a930b4fc0c5c79c7fd583201f7">12&nbsp;077</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/lucien-martinez">Lucien Martinez</a></td><td>1993</td><td><a href="/fr/scorecard/classement/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">FRA</a></td><td><a id="CityUrl" href="/fr/scorecard/classement/?City=montauban&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">montauban</a></td>
			</tr><tr>
				<td>39</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/andre-neres/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=97fd395524c16bfd90b8c59c9ac5b078">12&nbsp;070</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/andre-neres">André Neres</a></td><td>1985</td><td><a href="/pt/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">PRT</a></td><td><a id="CityUrl" href="/pt/scorecard/ranking/?City=Lisboa&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Lisboa</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>40</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/laurent-hogan/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=7c2cebc0f04bef2fd24f4a25b9db2a87">12&nbsp;050</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/laurent-hogan">Laurent Hogan</a></td><td>1978</td><td><a href="/fr/scorecard/classement/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">FRA</a></td><td><a id="CityUrl" href="/fr/scorecard/classement/?City=Peypin+d%27aigues&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Peypin d'aigues</a></td>
			</tr><tr>
				<td>41</td><td>&nbsp;</td><td><a href="/scorecard/thomas-blaabjerg/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=cd950961af4a7e8443357e3ec9f96d1a">12&nbsp;050</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/thomas-blaabjerg">Thomas Blaabjerg</a></td><td>1995</td><td><a href="/dk/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DNK</a></td><td><a id="CityUrl" href="/dk/scorecard/ranking/?City=Aalborg&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Aalborg</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>42</td><td>&nbsp;</td><td><a href="/scorecard/alex-ventajas/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=542049e33f8ac6a048e14db74800da4e">12&nbsp;045</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/alex-ventajas">Alex Ventajas</a></td><td>1999</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=murcia&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">murcia</a></td>
			</tr><tr>
				<td>43</td><td>&nbsp;</td><td><a href="/scorecard/jan-vopat/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=47376c70fcac8338d75a7456b63a9592">12&nbsp;023</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jan-vopat">Jan Vopat</a></td><td>2003</td><td><a href="/cz/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CZE</a></td><td><a id="CityUrl" href="/cz/scorecard/ranking/?City=%c3%9ast%c3%ad+nad+Labem&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Ústí nad Labem</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>44</td><td>&nbsp;</td><td><a href="/scorecard/thomas-dauser/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=7fc1b415d8c5f4b440f8e9e0ff4e1bf7">11&nbsp;983</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/thomas-dauser">Thomas Dauser</a></td><td>1987</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Bamberg&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Bamberg</a></td>
			</tr><tr>
				<td>45</td><td>&nbsp;</td><td><a href="/scorecard/kymy-de-la-pena/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=6fa9efa0fa1ed872e0cbfe1343894449">11&nbsp;980</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/kymy-de-la-pena">KYMY DE LA PEÑA</a></td><td>1985</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=madrid&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">madrid</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>46</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/stefano-folgarait/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=8b4a66d067e8ca89f9138d5384130567">11&nbsp;980</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/stefano-folgarait">Stefano Folgarait</a></td><td>&nbsp;</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Rovereto&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Rovereto</a></td>
			</tr><tr>
				<td>47</td><td>&nbsp;</td><td><a href="/scorecard/toni-lamprecht-290/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=5d54af1d97f7c817e4ac73f45fed1f9f">11&nbsp;970</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/toni-lamprecht-290">Toni Lamprecht</a></td><td>1971</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Munich&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Munich</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>48</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/jaka-sprah/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=ca1bec31d8d3e65d9a642fa19d73cbf4">11&nbsp;965</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/jaka-sprah">Jaka Šprah</a></td><td>1999</td><td><a href="/si/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">SVN</a></td><td><a id="CityUrl" href="/si/scorecard/ranking/?City=Ljubljana&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Ljubljana</a></td>
			</tr><tr>
				<td>49</td><td>&nbsp;</td><td><a href="/scorecard/wojtek-pelka/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=8030e8b1760c71b8e5538bdc316abb58">11&nbsp;965</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/wojtek-pelka">Wojtek Pełka</a></td><td>1999</td><td><a href="/pl/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">POL</a></td><td><a id="CityUrl" href="/pl/scorecard/ranking/?City=Krak%c3%b3w&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Kraków</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>50</td><td>&nbsp;</td><td><a href="/scorecard/davide-picco/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=05e10d30eef2be7702044c96da7df885">11&nbsp;945</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/davide-picco">Davide Picco</a></td><td>1992</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Schio&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Schio</a></td>
			</tr><tr>
				<td>51</td><td>&nbsp;</td><td><a href="/scorecard/stefan-scarperi/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=b4dea35ee03b4a26ea27d22f69228f1d">11&nbsp;908</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/stefan-scarperi">Stefan Scarperi</a></td><td>1991</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Appiano+(BZ)&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Appiano (BZ)</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>52</td><td>&nbsp;</td><td><a href="/scorecard/christian-muench/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=f9c64b662bdd37ed77e62f1dfeafd899">11&nbsp;890</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/christian-muench">Christian Münch</a></td><td>1988</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Munich&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Munich</a></td>
			</tr><tr>
				<td>53</td><td>&nbsp;</td><td><a href="/scorecard/davide-carena/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=d7bfadae8f21b3fd6d2d990c7956fb8b">11&nbsp;877</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/davide-carena">Davide Carena</a></td><td>2003</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Genova&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Genova</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>54</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/vonarburg-nicola/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=4118f65f1df71c13818aa6181bff9d13">11&nbsp;875</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/vonarburg-nicola">Vonarburg Nicola</a></td><td>1972</td><td><a href="/ch/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CHE</a></td><td><a id="CityUrl" href="/ch/scorecard/ranking/?City=Mendrisio&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Mendrisio</a></td>
			</tr><tr>
				<td>55</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/klemen-novak/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=a155bc41fba40f0b0799580778b7b849">11&nbsp;870</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/klemen-novak">Klemen Novak</a></td><td>2000</td><td><a href="/si/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">SVN</a></td><td><a id="CityUrl" href="/si/scorecard/ranking/?City=Kranj&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Kranj</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>56</td><td>&nbsp;</td><td><a href="/scorecard/lorenzo-bogliacino/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=bb6e9b759b4fb061f1f22b99f654b09e">11&nbsp;870</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/lorenzo-bogliacino">Lorenzo Bogliacino</a></td><td>1997</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Carcare&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Carcare</a></td>
			</tr><tr>
				<td>57</td><td>&nbsp;</td><td><a href="/scorecard/jonas-junker/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=555977fade56735da7a34b6b55a01f50">11&nbsp;840</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jonas-junker">Jonas Junker</a></td><td>&nbsp;</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=M%c3%bcnchen&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">München</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>58</td><td>&nbsp;</td><td><a href="/scorecard/michaela-kiersch/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=5b5ab01e154669825bd61a8e5f4e7695">11&nbsp;833</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/michaela-kiersch">Michaela Kiersch</a></td><td>1994</td><td><a href="/us/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">USA</a></td><td><a id="CityUrl" href="/us/scorecard/ranking/?City=Chicago&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Chicago</a></td>
			</tr><tr>
				<td>59</td><td>&nbsp;</td><td><a href="/scorecard/sascha-lehmann/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=f9e2e8fba00227c2bb52d8b328fa6bd8">11&nbsp;811</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/sascha-lehmann">Sascha Lehmann</a></td><td>&nbsp;</td><td><a href="/ch/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CHE</a></td><td><a id="CityUrl" href="/ch/scorecard/ranking/?City=Burgdorf&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Burgdorf</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>60</td><td>&nbsp;</td><td><a href="/scorecard/nico-ferlitsch/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=008c9af037bb5ebe0fd8b4eea447a98d">11&nbsp;810</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/nico-ferlitsch">Nico Ferlitsch</a></td><td>2001</td><td><a href="/at/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">AUT</a></td><td><a id="CityUrl" href="/at/scorecard/ranking/?City=Hermagor&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Hermagor</a></td>
			</tr><tr>
				<td>61</td><td>&nbsp;</td><td><a href="/scorecard/clemens-berger/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=0755c55fd995ec57040cb7d30381e95f">11&nbsp;802</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/clemens-berger">Clemens Berger</a></td><td>1998</td><td><a href="/at/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">AUT</a></td><td><a id="CityUrl" href="/at/scorecard/ranking/?City=Fieberbrunn&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Fieberbrunn</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>62</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/alex-espejo-saavedra/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=b9dc7972e8c8030f8c1c27e320e958f4">11&nbsp;798</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/alex-espejo-saavedra">Alex Espejo-Saavedra</a></td><td>1985</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=Malaga&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Malaga</a></td>
			</tr><tr>
				<td>63</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/jakob-bizjak/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=17f7c16de7cd1c3a704bbcac61806166">11&nbsp;790</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jakob-bizjak">Jakob Bizjak</a></td><td>1991</td><td><a href="/si/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">SVN</a></td><td><a id="CityUrl" href="/si/scorecard/ranking/?City=Roga%c2%9aka+slatina&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Rogaka slatina</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>64</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/moi-barea/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=65f6e18a2699d26f6f0ba24aac60f561">11&nbsp;785</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/moi-barea">Moi Barea</a></td><td>1982</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=Huelva&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Huelva</a></td>
			</tr><tr>
				<td>65</td><td>&nbsp;</td><td><a href="/scorecard/marcin-wszolek/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=ffe1cbcd1440259616f8979294638339">11&nbsp;750</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/marcin-wszolek">Marcin Wszolek</a></td><td>1984</td><td><a href="/pl/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">POL</a></td><td><a id="CityUrl" href="/pl/scorecard/ranking/?City=Krak%c3%b3w&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Kraków</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>66</td><td>&nbsp;</td><td><a href="/scorecard/bj-tilden/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=b6c347dfd980b72b8b1240a233fc7877">11&nbsp;740</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/bj-tilden">BJ Tilden</a></td><td>1980</td><td><a href="/us/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">USA</a></td><td><a id="CityUrl" href="/us/scorecard/ranking/?City=Lander&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Lander</a></td>
			</tr><tr>
				<td>67</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/christoph-kappacher/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=164d367386ac3b1fd98a63b8e9697df8">11&nbsp;740</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/christoph-kappacher">Christoph Kappacher</a></td><td>1992</td><td><a href="/at/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">AUT</a></td><td><a id="CityUrl" href="/at/scorecard/ranking/?City=Innsbruck&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Innsbruck</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>68</td><td>&nbsp;</td><td><a href="/scorecard/dru-mack-46843/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=d084dbccd445ea0a9656c444376b4707">11&nbsp;730</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/dru-mack-46843">Dru Mack</a></td><td>1994</td><td><a href="/us/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">USA</a></td><td><a id="CityUrl" href="/us/scorecard/ranking/?City=Pewee+Valley&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Pewee Valley</a></td>
			</tr><tr>
				<td>69</td><td>&nbsp;</td><td><a href="/scorecard/stephan-vogt/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=2d8c1c1bd9f452ace7319f4307352edb">11&nbsp;725</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/stephan-vogt">Stephan Vogt</a></td><td>1994</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Berlin&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Berlin</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>70</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/alessandro-larcher/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=7267595426d16068505f6d904692e334">11&nbsp;720</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/alessandro-larcher">Alessandro Larcher</a></td><td>1999</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Trento&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Trento</a></td>
			</tr><tr>
				<td>71</td><td>&nbsp;</td><td><a href="/scorecard/alberto-gines-lopez/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=b6bed8528bffb953ffd5783630c2513d">11&nbsp;720</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/alberto-gines-lopez">Alberto Gines Lopez</a></td><td>2002</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=Caceres&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Caceres</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>72</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/filip-schenk/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=215ba6c51ea2fab604319fe3b63b0140">11&nbsp;703</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/filip-schenk">Filip Schenk</a></td><td>2000</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=St.Christina&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">St.Christina</a></td>
			</tr><tr>
				<td>73</td><td>&nbsp;</td><td><a href="/scorecard/roland-wagner/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=d0286fe6d6a079c967bf0afcdcbbe7b0">11&nbsp;700</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/roland-wagner">Roland Wagner</a></td><td>1979</td><td><a href="/at/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">AUT</a></td><td><a id="CityUrl" href="/at/scorecard/ranking/?City=Salzburg&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Salzburg</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>74</td><td>&nbsp;</td><td><a href="/scorecard/dan-beland/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=39fc46479cf6aeb570e02e825095a1d8">11&nbsp;700</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/dan-beland">Dan Beland</a></td><td>1985</td><td><a href="/ca/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CAN</a></td><td><a id="CityUrl" href="/ca/scorecard/ranking/?City=Victoria&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Victoria</a></td>
			</tr><tr>
				<td>75</td><td>&nbsp;</td><td><a href="/scorecard/katherine-choong/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=4574cb59bdacafee0ad369c9eef17e3c">11&nbsp;700</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/katherine-choong">katherine choong</a></td><td>&nbsp;</td><td><a href="/ch/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CHE</a></td><td><a id="CityUrl" href="/ch/scorecard/ranking/?City=Del%c3%a9mont&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Delémont</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>76</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/lucas-de-jesus-martin/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=f4532488ca25a803030f2d63c8d9ace1">11&nbsp;695</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/lucas-de-jesus-martin">lucas de jesus martin</a></td><td>1996</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=San+Sebastian+de+los+Reyes&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">San Sebastian de los Reyes</a></td>
			</tr><tr>
				<td>77</td><td>&nbsp;</td><td><a href="/scorecard/jared-nelson/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=54a11fa8d43d9e9ed771d42cab7d0fad">11&nbsp;695</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jared-nelson">Jared Nelson</a></td><td>&nbsp;</td><td><a href="/ca/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CAN</a></td><td><a id="CityUrl" href="/ca/scorecard/ranking/?City=Kitchener&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Kitchener</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>78</td><td>&nbsp;</td><td><a href="/scorecard/pietro-radassao/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=ddf8abdf6e3c32e59513ce2faa7b533e">11&nbsp;682</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/pietro-radassao">Pietro Radassao</a></td><td>1995</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Campobasso&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Campobasso</a></td>
			</tr><tr>
				<td>79</td><td>&nbsp;</td><td><a href="/scorecard/alan-luna-69549/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=fa953cdf0c2cc0d25ded9f0836e2d58e">11&nbsp;680</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/alan-luna-69549">ALAN LUNA</a></td><td>1987</td><td><a href="/mx/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">MEX</a></td><td><a id="CityUrl" href="/mx/scorecard/ranking/?City=MEXICO+CITY&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">MEXICO CITY</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>80</td><td>&nbsp;</td><td><a href="/scorecard/guido-princess/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=d39fd41da5c8016fec55e02156d66d01">11&nbsp;675</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/guido-princess">Guido Princess</a></td><td>1989</td><td><a href="/us/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">USA</a></td><td><a id="CityUrl" href="/us/scorecard/ranking/?City=Fayetteville&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Fayetteville</a></td>
			</tr><tr>
				<td>81</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/josh-wharton/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=601ea8c01e70537a7afc66b336e1859e">11&nbsp;665</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/josh-wharton">josh wharton</a></td><td>&nbsp;</td><td><a href="/us/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">USA</a></td><td><a id="CityUrl" href="/us/scorecard/ranking/?City=estes+park&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">estes park</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>82</td><td>&nbsp;</td><td><a href="/scorecard/denis-pail/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=96d8c946fcf5c088c380ec8c85a353b4">11&nbsp;652</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/denis-pail">Denis Pail</a></td><td>1996</td><td><a href="/cz/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">CZE</a></td><td><a id="CityUrl" href="/cz/scorecard/ranking/?City=Ostrava&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Ostrava</a></td>
			</tr><tr>
				<td>83</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/giuseppe-nolasco/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=c2fdf0d662bac7f2983595d516011dff">11&nbsp;643</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/giuseppe-nolasco">Giuseppe Nolasco</a></td><td>1984</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Guardabosone&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Guardabosone</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>84</td><td>&nbsp;</td><td><a href="/scorecard/manu-lopez-6726/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=5cb89156628081057d327c3a1546c0bf">11&nbsp;643</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/manu-lopez-6726">Manu Lopez</a></td><td>1988</td><td><a href="/fr/scorecard/classement/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">FRA</a></td><td><a id="CityUrl" href="/fr/scorecard/classement/?City=Oloron&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Oloron</a></td>
			</tr><tr>
				<td>85</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/uli-fernandez/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=c16166981343ccbbda49a4185cec57af">11&nbsp;640</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/uli-fernandez">Uli Fernandez</a></td><td>1995</td><td><a href="/mx/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">MEX</a></td><td><a id="CityUrl" href="/mx/scorecard/ranking/?City=Guadalajara&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Guadalajara</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>86</td><td>&nbsp;</td><td><a href="/scorecard/grzegorz-sowa/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=e470c38b642edfd693cedcd46c66c7b5">11&nbsp;635</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/grzegorz-sowa">Grzegorz Sowa</a></td><td>1975</td><td><a href="/pl/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">POL</a></td><td><a id="CityUrl" href="/pl/scorecard/ranking/?City=Zakopane&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Zakopane</a></td>
			</tr><tr>
				<td>87</td><td>&nbsp;</td><td><a href="/scorecard/tobias-haug/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=7b9dbd93b9a3f82b4eea4db2a2007a93">11&nbsp;635</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/tobias-haug">Tobias Haug</a></td><td>1973</td><td><a href="/tr/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">TUR</a></td><td><a id="CityUrl" href="/tr/scorecard/ranking/?City=Citdibi&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Citdibi</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>88</td><td>&nbsp;</td><td><a href="/scorecard/mani-hubaer/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=848b2be4f0be1da9f977f5df82acda1f">11&nbsp;633</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/mani-hubaer">Mani Hubär</a></td><td>1990</td><td><a href="/at/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">AUT</a></td><td><a id="CityUrl" href="/at/scorecard/ranking/?City=Vienna&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Vienna</a></td>
			</tr><tr>
				<td>89</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/hernan-garcia/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=32f21826537cce02b455b48e63b004ec">11&nbsp;615</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/hernan-garcia">Hernan Garcia</a></td><td>1982</td><td><a href="/mx/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">MEX</a></td><td><a id="CityUrl" href="/mx/scorecard/ranking/?City=Mexico.+DF.&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Mexico. DF.</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>90</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/andrea-daddazio/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=c69800765a64d53c0ec873362023b544">11&nbsp;610</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/andrea-daddazio">Andrea D'Addazio</a></td><td>1999</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Penne&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Penne</a></td>
			</tr><tr>
				<td>91</td><td>&nbsp;</td><td><a href="/scorecard/erik-libbe/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=6376dfa2cdb3bdc4c216c8f06bda4576">11&nbsp;603</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/erik-libbe">Erik Libbe</a></td><td>&nbsp;</td><td><a href="/us/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">USA</a></td><td><a id="CityUrl" href="/us/scorecard/ranking/?City=Ann+Arbor&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Ann Arbor</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>92</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/rafa-fanega/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=57d75005b27a0d9550748cbe012c4238">11&nbsp;595</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/rafa-fanega">Rafa Fanega</a></td><td>1975</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=Madrid&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Madrid</a></td>
			</tr><tr>
				<td>93</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/robert-de-lafondue/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=4a9fadaade50f202513c062120130aed">11&nbsp;595</a></td><td><img src="/images/common/onLine8a.gif" alt="On Line"></td><td><a href="/user/robert-de-lafondue">Robert De lafondue</a></td><td>1994</td><td><a href="/fr/scorecard/classement/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">FRA</a></td><td><a id="CityUrl" href="/fr/scorecard/classement/?City=Roland+Garros&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Roland Garros</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>94</td><td>&nbsp;</td><td><a href="/scorecard/jakob-greistorfer/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=7c0e31a0717c2489b589576a2afd71dc">11&nbsp;592</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jakob-greistorfer">Jakob Greistorfer</a></td><td>1994</td><td><a href="/at/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">AUT</a></td><td><a id="CityUrl" href="/at/scorecard/ranking/?City=Graz&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Graz</a></td>
			</tr><tr>
				<td>95</td><td>&nbsp;</td><td><a href="/scorecard/tomatis-giorgio/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=f2887cb9d710dd0d36720da217ac3790">11&nbsp;573</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/tomatis-giorgio">Tomatis Giorgio</a></td><td>&nbsp;</td><td><a href="/it/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ITA</a></td><td><a id="CityUrl" href="/it/scorecard/ranking/?City=Caraglio&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Caraglio</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>96</td><td>&nbsp;</td><td><a href="/scorecard/adam-gadula-karpierz/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=2fe5f1ddf0996cbcb2649a010cc13196">11&nbsp;555</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/adam-gadula-karpierz">Adam "Gaduła" Karpierz</a></td><td>1991</td><td><a href="/pl/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">POL</a></td><td><a id="CityUrl" href="/pl/scorecard/ranking/?City=Krak%c3%b3w&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Kraków</a></td>
			</tr><tr>
				<td>97</td><td>&nbsp;</td><td><a href="/scorecard/michael-schreiber/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=c03ba747b75513883d09c6bf495b9e5f">11&nbsp;550</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/michael-schreiber">Michael Schreiber</a></td><td>1987</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=Berlin&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Berlin</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>98</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/solveig-korherr/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=ad2dcf407739fea0fb77debb2b8ddc37">11&nbsp;535</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/solveig-korherr">Solveig Korherr</a></td><td>1998</td><td><a href="/de/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">DEU</a></td><td><a id="CityUrl" href="/de/scorecard/ranking/?City=&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1"></a></td>
			</tr><tr>
				<td>99</td><td><img src="/images/common/rankingArrowUp.gif" alt="Ascents in last month"></td><td><a href="/scorecard/jean-rajoute/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=94a8fa19078ff928792ca3038fff4f2e">11&nbsp;534</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/jean-rajoute">jean rajoute</a></td><td>&nbsp;</td><td><a href="/fr/scorecard/classement/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">FRA</a></td><td><a id="CityUrl" href="/fr/scorecard/classement/?City=new+york&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">new york</a></td>
			</tr><tr style="background-color:WhiteSmoke;">
				<td>100</td><td>&nbsp;</td><td><a href="/scorecard/iris-matamoros-quero/routes/?AscentClass=0&amp;AscentListTimeInterval=1&amp;AscentListViewType=0&amp;GID=980dfd4f5a4a4c5508798bbc295658cf">11&nbsp;533</a></td><td><img src="/images/common/offLine8a.gif" alt="Off Line"></td><td><a href="/user/iris-matamoros-quero">Iris Matamoros Quero</a></td><td>1980</td><td><a href="/es/scorecard/ranking/?RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">ESP</a></td><td><a id="CityUrl" href="/es/scorecard/ranking/?City=Murcia&amp;RankingAgeLimit=0&amp;CombinedRanking=0&amp;RankingListType=1&amp;Gender=-1">Murcia</a></td>
			</tr><tr>
				<td colspan="8"><table border="0">
					<tbody><tr>
						<td><a href="javascript:__doPostBack('ctl00$ContentPlaceholder$GridViewRankingRoute','Page$Next')">Next 100</a></td>
					</tr>
				</tbody></table></td>
			</tr>
		</tbody></table>

# Web scrape

In [2]:
re_scrape = False
if re_scrape:
    url_8a = 'https://www.8a.nu/scorecard/ranking/'
    driver = webdriver.Chrome(executable_path='./chromedriver')

    # 8a.nu is incredibly slow, so we're going to be doing this a lot
    driver.get(url_8a)
    driver.implicitly_wait(30)

    dfs = {'RankingRoute': None,
          'Boulder': None}

    for sport in dfs.keys():
        df = pd.DataFrame()
        for i in range(15):

            time.sleep(3)
            soup=BeautifulSoup(driver.page_source, "html.parser")
            table_id = 'ctl00_ContentPlaceholder_GridView' + sport
            table1 = soup.find('table', id=table_id)
            time.sleep(3)
            new_df = pd.DataFrame(pd.read_html(str(table1))[0])
            time.sleep(5)
            next_button_path = "//a[contains(@href,'GridView"+sport+"') and contains(.,'Next')]"
            next_routes = driver.find_element_by_xpath(next_button_path)
            actions = ActionChains(driver)
            actions.move_to_element(next_routes).perform()
            time.sleep(3)
            next_routes.click()

            df = pd.concat([df, new_df], axis=0)

        df.columns = ['ranking', 'drop','points', 'drop1','name','drop2','drop3','drop4']
        df.drop([column for column in df.columns if 'drop' in column], axis=1, inplace=True)
        df.dropna(inplace=True)
        df['points'] = df['points'].replace(r'\xa0', '', regex=True)
        df = df[~df.points.str.contains("Next 100")]
        df['points'] = df['points'].astype(int)
        dfs[sport] = df

    print('\a\a\a done')
    
else:
    with open('8a_data.pkl', 'rb') as handle:
        dfs = pickle.load(handle)

In [ ]:
# Routes
trace1 = go.Scatter(
            x = dfs['RankingRoute']['ranking'],
            y = dfs['RankingRoute']['points'],
            name='routes',
            line = {'color': ('rgb(255,0,0)')})

route_tick_vals = [val for val in range(8500, 14500, 500)]
route_tick_labels = ['7B+','7C','7C+', '8A','8A+', '8B', '8B+', '8C', '8C+', '9A' ,'9A+', '9B','9B+']

# Boulders
trace2 = go.Scatter(
            x = dfs['Boulder']['ranking'],
            y = dfs['Boulder']['points'],
            name = 'boulders',
            yaxis = 'y2',
            line = {'color': ('rgb(0,0,255)')})

boulder_tick_vals = [val for val in range(7000,12500, 500)]
boulder_tick_labels = ['v5','v6','v7','v8', 'v9', 'v10', 'v11', 'v12','v13','v14','v15', 'v16', 'v17', 'v18']


data = [trace1, trace2]

layout = go.Layout(
            title = 'Comparison of Points by 8a Ranking',
            xaxis = {'title': 'Ranking'},
            yaxis = {'title':'Route Points',
                     'tickvals': route_tick_vals,
                     'ticktext': route_tick_labels,
                     'range': [8500,14000],
                     'titlefont': {'color': 'rgb(255,0,0)'},
                     'tickfont': {'color': 'rgb(255,0,0)'},},
    
            yaxis2 = {'title': 'Bouldering Points',
                      
                     'tickvals': boulder_tick_vals,
                     'ticktext': boulder_tick_labels,
                     'titlefont': {'color': 'rgb(0,0,255)'},
                     'tickfont': {'color': 'rgb(0,0,255)'},
                     'overlaying': 'y',
                     'side': 'right',
                     'range': [7000,12500]}
                    )

fig = go.Figure(data=data, layout=layout)
plot_url = plot(fig)

In [ ]:
# I don't want to do this by accident, and I don't want to accidentaly uncomment it.
# don't look at me

if False:
    with open('8a_data.pkl', 'wb') as pickle_file:
        pickle.dump(dfs, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36'

names = dfs['RankingRoute'].name.values
def scrap_bmi_data(names):
    names = [name.lower().replace(' ','-') for name in names]
    urls = ['https://www.8a.nu/user/'+name for name in names]
    heights = []
    weights = []
    http = urllib3.PoolManager()
    for url in urls:
        url = urllib.parse.urlsplit(url)
        url = list(url)
        url[2] = urllib.parse.quote(url[2])
        url = urllib.parse.urlunsplit(url)
        page = http.request('GET', url)
        print(url)
        response = get(url, headers={'User-Agent':user_agent}, allow_redirects=True)

        soup = BeautifulSoup(response.text, 'html.parser')
        spans = soup('span')
        try:
            height = soup.find("td", text="Height").find_next_sibling("td").text
            print(height)
            if height == '':
                height = None
            else:
                height = int(height.split(' ')[0])
            heights.append(height)

            weight = soup.find('td', text='Weight').find_next_sibling("td").text
            print(weight)
            if weight =='':
                weight = None
            else:
                weight = int(weight.split(' ')[1])
            weights.append(weight)

        except AttributeError:
            heights.append(None)
    
    return heights, weights

In [ ]:
names = dfs['RankingRoute'].name.values
route_heights, route_weights = scrap_bmi_data(names)

names = dfs['Boulder'].name.values
boulder_heights, boulder_weights = scrap_bmi_data(names)

https://www.8a.nu/user/adam-ondra
185 cm
about 68 kg
https://www.8a.nu/user/piotr-schab
173 cm
about 58 kg
https://www.8a.nu/user/jonathan-siegrist
167 cm
about 58 kg
https://www.8a.nu/user/pirmin-bertle
184 cm
about 68 kg
https://www.8a.nu/user/jorge-diaz-rullo
https://www.8a.nu/user/mathieu-bouyoud
180 cm
about 63 kg
https://www.8a.nu/user/david-firnenburg
181 cm

https://www.8a.nu/user/stefano-ghisolfi
170 cm
about 58 kg
https://www.8a.nu/user/alex-garriga
172 cm
about 53 kg
https://www.8a.nu/user/moritz-welt
176 cm
about 63 kg
https://www.8a.nu/user/ram%C3%B3n-julian-puigblanque
158 cm
about 48 kg
https://www.8a.nu/user/thibault-lair
169 cm
about 63 kg
https://www.8a.nu/user/jernej-kruder
180 cm
about 68 kg
https://www.8a.nu/user/luca-bana
170 cm
about 58 kg
https://www.8a.nu/user/philipp-ga%C3%9Fner
https://www.8a.nu/user/marco-zanone
185 cm
about 73 kg
https://www.8a.nu/user/jan-hojer
186 cm

https://www.8a.nu/user/evan-hau


https://www.8a.nu/user/peter-dawson


https://www.8a.n

https://www.8a.nu/user/ema-seli%C5%A1kar
160 cm
about 48 kg
https://www.8a.nu/user/apple-steff


https://www.8a.nu/user/david-galan
169 cm
about 58 kg
https://www.8a.nu/user/daniel-m%C3%BCller
https://www.8a.nu/user/illya-bakhmet-smolenskyi
138 cm
about 40 kg
https://www.8a.nu/user/cristos-daniil
186 cm

https://www.8a.nu/user/tim-reuser
182 cm
about 73 kg
https://www.8a.nu/user/ben-herrington
182 cm
about 68 kg
https://www.8a.nu/user/moritz-sigmund
178 cm
about 58 kg
https://www.8a.nu/user/paul-steinig
167 cm
about 53 kg
https://www.8a.nu/user/martina-harnisch-%28f%29
https://www.8a.nu/user/alex-herbert
183 cm
about 73 kg
https://www.8a.nu/user/ben-west
182 cm
about 73 kg
https://www.8a.nu/user/octavio-hernandez
175 cm
about 63 kg
https://www.8a.nu/user/alvaro-lafuente
179 cm
about 68 kg
https://www.8a.nu/user/lohan-lizin
177 cm
about 63 kg
https://www.8a.nu/user/lucas-marques

about 73 kg
https://www.8a.nu/user/rafal-bak
146 cm
about 48 kg
https://www.8a.nu/user/luca-de-demo
177 cm
a

168 cm
about 58 kg
https://www.8a.nu/user/kensie-whitfield

about 73 kg
https://www.8a.nu/user/guillaume-gomord
192 cm
about 78 kg
https://www.8a.nu/user/andreas-hanisch
175 cm

https://www.8a.nu/user/olga-niemiec
158 cm

https://www.8a.nu/user/matt-pincus

about 68 kg
https://www.8a.nu/user/c.-burkhart
https://www.8a.nu/user/alejandro-bada
174 cm
about 63 kg
https://www.8a.nu/user/andy-winterleitner
164 cm

https://www.8a.nu/user/nick-lozica
180 cm
about 63 kg
https://www.8a.nu/user/jon-shen
https://www.8a.nu/user/ally-smith
185 cm
about 78 kg
https://www.8a.nu/user/carlos-zanon


https://www.8a.nu/user/sami-aziz
188 cm
about 63 kg
https://www.8a.nu/user/steve-luccarda


https://www.8a.nu/user/jorg-m
182 cm
about 73 kg
https://www.8a.nu/user/helena-alem%C3%A1n-%28f%29
https://www.8a.nu/user/aristomenis-thanopoulos
175 cm
about 63 kg
https://www.8a.nu/user/nick-churchill
185 cm
about 73 kg
https://www.8a.nu/user/beaulieu-jeff
171 cm
about 68 kg
https://www.8a.nu/user/zeno-schaekers
178

145 cm
about 40 kg
https://www.8a.nu/user/emanuele-quaglia
176 cm
about 53 kg
https://www.8a.nu/user/nick-orange
170 cm
about 68 kg
https://www.8a.nu/user/jillian-sompel
https://www.8a.nu/user/vladimir-gvozdev
175 cm
about 63 kg
https://www.8a.nu/user/matic-obid


https://www.8a.nu/user/afroben-b
173 cm
about 68 kg
https://www.8a.nu/user/charlie-egan


https://www.8a.nu/user/eline-n%C3%A6sheim
https://www.8a.nu/user/philipp-wellmann
187 cm
about 73 kg
https://www.8a.nu/user/brian-weaver
173 cm
about 68 kg
https://www.8a.nu/user/lucy-mitchell


https://www.8a.nu/user/adam-macke
169 cm
about 68 kg
https://www.8a.nu/user/katarina-rus-%28f%29
https://www.8a.nu/user/yury-shamardin
172 cm
about 58 kg
https://www.8a.nu/user/tancredi-cappa
170 cm
about 68 kg
https://www.8a.nu/user/serpa-pastor
170 cm
about 63 kg
https://www.8a.nu/user/juan-sebasti%C3%A1n-arboleda
167 cm
about 58 kg
https://www.8a.nu/user/edwin-teran


https://www.8a.nu/user/tilo-moser


https://www.8a.nu/user/david-coliseo
htt

176 cm
about 68 kg
https://www.8a.nu/user/vroni-ga%C3%9Fner
https://www.8a.nu/user/nadja-pfister


https://www.8a.nu/user/une-poule
https://www.8a.nu/user/quinn-mason
170 cm

https://www.8a.nu/user/pascal-bernhard
180 cm

https://www.8a.nu/user/lynn-van-der-meer


https://www.8a.nu/user/marco-ferrario
162 cm
about 53 kg
https://www.8a.nu/user/kuba-robakowski
https://www.8a.nu/user/marcel-dettling
188 cm
about 78 kg
https://www.8a.nu/user/aleksandra-taistra-%28f%29
https://www.8a.nu/user/vladislav-angelov
180 cm
about 78 kg
https://www.8a.nu/user/fred-beguec
180 cm

https://www.8a.nu/user/eric-destefanis
178 cm
about 73 kg
https://www.8a.nu/user/nico-las


https://www.8a.nu/user/matteo-calledda
180 cm
about 63 kg
https://www.8a.nu/user/kuba-szumi%C5%82o
https://www.8a.nu/user/larissa-arce
167 cm
about 48 kg
https://www.8a.nu/user/natalie-berry
100 cm
about 100 kg
https://www.8a.nu/user/kuba-polak
190 cm
about 83 kg
https://www.8a.nu/user/jay-koh
170 cm
about 53 kg
https://www.8a.nu/user

...



https://www.8a.nu/user/michael-molony


https://www.8a.nu/user/david-rodriguez
175 cm
about 63 kg
https://www.8a.nu/user/kusiek-tataraaa
182 cm
about 63 kg
https://www.8a.nu/user/patryk-r
170 cm
about 68 kg
https://www.8a.nu/user/evan-wells
https://www.8a.nu/user/marcony-marconi


https://www.8a.nu/user/frances-bensley
165 cm
about 53 kg
https://www.8a.nu/user/martijn-collenteur
175 cm

https://www.8a.nu/user/timo-kienzle


https://www.8a.nu/user/jan-erik-rapp


https://www.8a.nu/user/lena-brandner
https://www.8a.nu/user/ela-pawelczyk


https://www.8a.nu/user/carlos-sol%C3%A9
https://www.8a.nu/user/ryan-%22k%C2%A2/b%C2%A2%22-%24urface
https://www.8a.nu/user/yves-schartz
178 cm
about 68 kg
https://www.8a.nu/user/gabriel-faucher
175 cm
about 73 kg
https://www.8a.nu/user/aim-forty


https://www.8a.nu/user/spiggy-6
https://www.8a.nu/user/ga%C2%9Aper-pintar
https://www.8a.nu/user/immanuel-diener
170 cm
about 58 kg
https://www.8a.nu/user/erwin-fedasz
169 cm

https://www.8a.nu/user/marko-st

167 cm
about 58 kg
https://www.8a.nu/user/bajusz-bal%C3%A1zs
184 cm
about 73 kg
https://www.8a.nu/user/martin-%C5%A1vec
185 cm
about 73 kg
https://www.8a.nu/user/toni-lamprecht


https://www.8a.nu/user/sam-mcqueen
177 cm
about 68 kg
https://www.8a.nu/user/alex-puccio-%28f%29
https://www.8a.nu/user/luis-serrano
166 cm

https://www.8a.nu/user/nick-bradley
157 cm
about 48 kg
https://www.8a.nu/user/mike-mcclure


https://www.8a.nu/user/luis-gerhardt
175 cm
about 58 kg
https://www.8a.nu/user/sam-sommers
175 cm
about 73 kg
https://www.8a.nu/user/zach-wilson
170 cm
about 43 kg
https://www.8a.nu/user/mattias-braach-maksvytis
186 cm
about 83 kg
https://www.8a.nu/user/josh-patzner
162 cm
about 48 kg
https://www.8a.nu/user/chase-gatland
179 cm
about 73 kg
https://www.8a.nu/user/gustavo-fontes
173 cm
about 63 kg
https://www.8a.nu/user/everett-sloane


https://www.8a.nu/user/enrico-lovato
171 cm
about 58 kg
https://www.8a.nu/user/tam%C3%A1s-farkas


https://www.8a.nu/user/ben-west
182 cm
about 73 k



https://www.8a.nu/user/max-kleesattel
172 cm

https://www.8a.nu/user/carlos-garc%C3%ADa-sierra
168 cm
about 43 kg
https://www.8a.nu/user/matthias-woidneck
182 cm
about 73 kg
https://www.8a.nu/user/ameen-hosain


https://www.8a.nu/user/matt-fowls
182 cm
about 73 kg
https://www.8a.nu/user/thomas-salakenos
175 cm
about 58 kg
https://www.8a.nu/user/christoph-reichert
183 cm
about 73 kg
https://www.8a.nu/user/ken-peterson


https://www.8a.nu/user/moritz-sigmund
178 cm
about 58 kg
https://www.8a.nu/user/laurent-hogan
177 cm
about 73 kg
https://www.8a.nu/user/nick-churchill
185 cm
about 73 kg
https://www.8a.nu/user/billy-ridal


https://www.8a.nu/user/zoltan-komjati
186 cm
about 73 kg
https://www.8a.nu/user/alexander-laure
https://www.8a.nu/user/marco-zanone
185 cm
about 73 kg
https://www.8a.nu/user/kody-shutt


https://www.8a.nu/user/jacopo-piatto


https://www.8a.nu/user/brandon-gezel
170 cm
about 63 kg
https://www.8a.nu/user/francesco-morandi
175 cm
about 68 kg
https://www.8a.nu/user/joh

183 cm
about 68 kg
https://www.8a.nu/user/maya-madere


https://www.8a.nu/user/luke-muehring

about 73 kg
https://www.8a.nu/user/zev-fineman


https://www.8a.nu/user/paul-winkler


https://www.8a.nu/user/maria-davies-sandbu-%28f%29
https://www.8a.nu/user/matt-gentile
165 cm
about 63 kg
https://www.8a.nu/user/ethan-hunt

about 63 kg
https://www.8a.nu/user/steven-augustine
https://www.8a.nu/user/samuel-sigouin
169 cm
about 58 kg
https://www.8a.nu/user/f-dave


https://www.8a.nu/user/p-shin
https://www.8a.nu/user/alex-knapp
172 cm
about 63 kg
https://www.8a.nu/user/charlie-schreiber
180 cm
about 63 kg
https://www.8a.nu/user/gian-visciano


https://www.8a.nu/user/guilherme-faoro
179 cm
about 63 kg
https://www.8a.nu/user/hank-gaylord
172 cm
about 63 kg
https://www.8a.nu/user/jonathan-woods
182 cm
about 78 kg
https://www.8a.nu/user/michael-forney
176 cm
about 63 kg
https://www.8a.nu/user/marcin-zwir


https://www.8a.nu/user/matt-norgrove
178 cm
about 63 kg
https://www.8a.nu/user/newton-neto


170 cm
about 53 kg
https://www.8a.nu/user/ethan-micele
175 cm
about 58 kg
https://www.8a.nu/user/chad-haarmed
https://www.8a.nu/user/drexel-bakker
188 cm
about 73 kg
https://www.8a.nu/user/birkir-fannar-sn%C3%A6varsson
https://www.8a.nu/user/harold-ainsworth
171 cm
about 53 kg
https://www.8a.nu/user/josh-horsley


https://www.8a.nu/user/derrek-chu
176 cm
about 63 kg
https://www.8a.nu/user/erik-jenson


https://www.8a.nu/user/jordan-shackelford


https://www.8a.nu/user/sam-wu
170 cm
about 58 kg
https://www.8a.nu/user/andreas-l%C3%B6hr
https://www.8a.nu/user/lando-peters
179 cm
about 73 kg
https://www.8a.nu/user/matt-yeow
175 cm
about 68 kg
https://www.8a.nu/user/seba-%22seggiola%22-montecucco
https://www.8a.nu/user/mike-wohner
178 cm
about 73 kg
https://www.8a.nu/user/paolini-yohann
173 cm
about 58 kg
https://www.8a.nu/user/cody-rawls
177 cm
about 68 kg
https://www.8a.nu/user/h%C3%A5kon-nordstr%C3%B8m
https://www.8a.nu/user/tara-hayes
169 cm
about 58 kg
https://www.8a.nu/user/held-der-a

In [8]:
dfs.keys()

dict_keys(['RankingRoute', 'Boulder'])